دیتاست تمیز و پیش‌پردازش شده

In [ ]:
import pandas as pd
!gdown --id <1Np9nenPVV0LLlsE909X7F3PN4nMES3YV> --output cleaned_data.csv
df_clean = pd.read_csv('cleaned_data.csv')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_clean.head()

,Row,Year,RemittanceDate,BirthYear,PersonnelCode,PlanName,DiseaseDate,ClaimDate,DiseaseType,Z_Log_Reported,...,Relation_0,Relation_1,Relation_2,Relation_3,Relation_4,Relation_5,Age,Age_Standardized,PlanName_Standardized,DiseaseType_Encoded
0,2,1401-1402,1401-09-10,1363,461415.0,4,1401-09-05,1401-09-06,هزینه ویزیت,-0.593003,...,False,True,False,False,False,False,41,0.246071,0.696407,-0.422600
1,3,1401-1402,1401-09-13,1340,461421.0,4,1401-09-03,1401-09-11,سایر اعمال مجاز سرپایی,0.913197,...,False,False,False,False,True,False,64,1.472632,0.696407,0.341211
2,4,1401-1402,1401-09-13,1340,461421.0,4,1401-09-03,1401-09-11,انواع آندوسکوپی و کولونوسکوپی,1.598488,...,False,False,False,False,True,False,64,1.472632,0.696407,1.370494
3,5,1401-1402,1401-09-13,1340,461421.0,4,1401-09-03,1401-09-11,هزینه ویزیت,-1.359214,...,False,False,False,False,True,False,64,1.472632,0.696407,-0.422600
4,6,1401-1402,1401-09-13,1340,461421.0,4,1401-09-03,1401-09-11,تزریقات,-1.239765,...,False,False,False,False,True,False,64,1.472632,0.696407,-1.327007


**از روی ستون «سال» سال پرداخت رو استخراج میکنم**

In [ ]:
def extract_first_year(year_range):
    if isinstance(year_range, str):
        return year_range.split('-')[0]
    return year_range

df_clean['FirstYear'] = df_clean['Year'].apply(extract_first_year)

In [ ]:
# استاندارد کردن ستون سال

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df_clean['FirstYear_Standardized'] = scaler.fit_transform(df_clean[['FirstYear']])

**رگرسیون خطی ساده**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# ویژگی‌ها
features = ['Z_Log_Reported', 'DiseaseType_Encoded',
            'Age_Standardized', 'PlanName_Standardized', 'FirstYear_Standardized']
features += [col for col in df_clean.columns if col.startswith('DiseaseGroup_') or col.startswith('Relation_') or col.startswith('Company_') or col.startswith('DocumentDeliveryType_')]

X = df_clean[features]
y = df_clean['Z_log_Paid']

# تقسیم داده‌ها
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ساخت و فیت مدل
model = LinearRegression()
model.fit(X_train, y_train)

# پیش‌بینی و ارزیابی
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
y_train_pred = model.predict(X_train)
r2_train = r2_score(y_train, y_train_pred)

print(f"Mean squared error: {mse}")
print(f"Model accuracy (R²) on training data: {r2_train}")
print(f"Model accuracy (R²): {r2}")

Mean squared error: 0.059309479440667126
Model accuracy (R²) on training data: 0.9367031490249446
Model accuracy (R²): 0.9376751360590935


**رگرسیون  پولینومیال درجه 2**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

features = ['Z_Log_Reported', 'DiseaseType_Encoded',
            'Age_Standardized', 'PlanName_Standardized', 'FirstYear_Standardized']
features += [col for col in df_clean.columns if col.startswith('DiseaseGroup_') or col.startswith('Relation_') or col.startswith('Company_') or col.startswith('DocumentDeliveryType_')]

X = df_clean[features]
y = df_clean['Z_log_Paid']

# تبدیل داده‌ها به چندجمله‌ای (درجه 2)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

# پیش‌بینی و ارزیابی
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
y_train_pred = model.predict(X_train)
r2_train = r2_score(y_train, y_train_pred)

print(f"Mean squared error: {mse}")
print(f"Model accuracy (R²) on training data: {r2_train}")
print(f"Model accuracy (R²): {r2}")

Mean squared error: 0.038477567322497574
Model accuracy (R²) on training data: 0.9596888388972263
Model accuracy (R²): 0.9595661743996458


**جنگل تصادفی**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

features = ['Z_Log_Reported', 'DiseaseType_Encoded',
            'Age_Standardized', 'PlanName_Standardized', 'FirstYear_Standardized']
features += [col for col in df_clean.columns if col.startswith('DiseaseGroup_') or col.startswith('Relation_') or col.startswith('Company_') or col.startswith('DocumentDeliveryType_')]

X = df_clean[features]
y = df_clean['Z_log_Paid']

# تقسیم داده‌ها
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# مدل Random Forest
model = RandomForestRegressor(n_estimators=200, max_depth=11, random_state=42)
model.fit(X_train, y_train)

# پیش‌بینی و ارزیابی
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
y_train_pred = model.predict(X_train)
r2_train = r2_score(y_train, y_train_pred)

print(f"Mean squared error: {mse}")
print(f"Model accuracy (R²) on training data: {r2_train}")
print(f"Model accuracy (R²): {r2}")

Mean squared error: 0.028131521018079704
Model accuracy (R²) on training data: 0.9748600635754799
Model accuracy (R²): 0.9704382294965755


**XGBOOST**

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# ویژگی‌ها
features = ['Z_Log_Reported', 'DiseaseType_Encoded',
            'Age_Standardized', 'PlanName_Standardized', 'FirstYear_Standardized']
features += [col for col in df_clean.columns if col.startswith('DiseaseGroup_') or col.startswith('Relation_') or col.startswith('Company_') or col.startswith('DocumentDeliveryType_')]

X = df_clean[features]
y = df_clean['Z_log_Paid']

# تقسیم داده‌ها
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# مدل XGBoost
model = XGBRegressor(
    n_estimators=240,
    max_depth=11,
    learning_rate=0.1,
    subsample=0.9,
    colsample_bytree=0.9,
    early_stopping_rounds=20,
    random_state=42
)

model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=False
)

# پیش‌بینی و ارزیابی
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
y_train_pred = model.predict(X_train)
r2_train = r2_score(y_train, y_train_pred)

print(f"Mean squared error: {mse}")
print(f"Model accuracy (R²) on training data: {r2_train}")
print(f"Model accuracy (R²): {r2}")

Mean squared error: 0.027015214676658506
Model accuracy (R²) on training data: 0.9807955737442162
Model accuracy (R²): 0.9716412415175868


In [ ]:
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
from xgboost import XGBRegressor

# Assuming X and y are already defined

# Create a KFold object for cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Define the XGBoost model without early stopping for cross-validation
model = XGBRegressor(
    n_estimators=300,
    max_depth=10,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    # Remove early_stopping_rounds for cross-validation
    random_state=42
)

# Perform cross-validation
scores = cross_val_score(model, X, y, scoring='r2', cv=kf, n_jobs=-1)

print("R² for each fold:", scores)
print("Average R²:", np.mean(scores))

R² for each fold: [0.97103032 0.97059012 0.97086534 0.97070658 0.97105392]
Average R²: 0.9708492553819221


**بررسی کراس‌ولیدیشن روی مدل تا ببینیم دچار اورفیت نشده باشد**

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=42)

model = XGBRegressor(
    n_estimators=240,
    max_depth=11,
    learning_rate=0.1,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42
)

r2_scores = cross_val_score(model, X, y, cv=kf, scoring='r2', n_jobs=-1)

print(f"Cross-Validation with R² average: {np.mean(r2_scores):.4f}")
print(f"Cross-Validation with R² standard deviation: {np.std(r2_scores):.4f}")

Cross-Validation with R² average: 0.9702
Cross-Validation with R² standard deviation: 0.0002


**چون میانگین کراس ولیدیشن نزدیک به دقت مدل است پس اورفیتی وجود ندارد**